In [50]:
import numpy as np
max_pad_len = 174

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [19]:
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../Capstone_Project_2/train/Train/'

metadata = pd.read_csv('../Capstone_Project_2/train_detail.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath)+'/',str(row["File_Name"]))
    
    class_label = row["Class"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

Finished feature extraction from  5435  files


In [20]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.3, random_state = 42)

In [26]:
x_train.shape

(174, 3804, 40, 1)

In [27]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[1], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[1], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [28]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [29]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 19, 86, 16)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 41, 64)        

In [31]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 72
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='../Capstone_Project_2/weights.best.basic_mlp.hdf6', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, 
          validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 3804 samples, validate on 1631 samples
Epoch 1/72
3804/3804 [==============================] - 12s 3ms/step - loss: 2.5155 - accuracy: 0.1798 - val_loss: 2.0692 - val_accuracy: 0.2287

Epoch 00001: val_loss improved from inf to 2.06924, saving model to ../Capstone_Project_2/weights.best.basic_mlp.hdf6
Epoch 2/72
3804/3804 [==============================] - 15s 4ms/step - loss: 2.0668 - accuracy: 0.2518 - val_loss: 1.9949 - val_accuracy: 0.3023

Epoch 00002: val_loss improved from 2.06924 to 1.99486, saving model to ../Capstone_Project_2/weights.best.basic_mlp.hdf6
Epoch 3/72
3804/3804 [==============================] - 20s 5ms/step - loss: 1.8708 - accuracy: 0.3273 - val_loss: 1.9036 - val_accuracy: 0.3648

Epoch 00003: val_loss improved from 1.99486 to 1.90362, saving model to ../Capstone_Project_2/weights.best.basic_mlp.hdf6
Epoch 4/72
3804/3804 [==============================] - 16s 4ms/step - loss: 1.6994 - accuracy: 0.3967 - val_loss: 1.7511 - val_accuracy: 0.4347

Epoch 

In [32]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9048370122909546
Testing Accuracy:  0.847946047782898


In [61]:
Class_Name = []
def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    
    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        Class_Name.append(category[0])

In [48]:
testdata = pd.read_csv('../Capstone_Project_2/test/test.csv')

In [56]:
x_test.shape

(1631, 40, 174, 1)

In [62]:
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../Capstone_Project_2/test/Test/'

testdata = pd.read_csv('../Capstone_Project_2/test/test.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in testdata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath)+'/',str(row["ID1"]))
    data = print_prediction(file_name)



In [63]:
Class=pd.DataFrame(Class_Name)

In [64]:
testdata['Class']=Class

In [65]:
testdata = testdata[['Class', 'ID1']]

In [69]:
testdata["ID1"]= testdata["ID1"].str.split(".", n = 1, expand = True) 


In [71]:
testdata.to_csv('test1.csv')